# Setup

In [1]:
from typing import Any
import json
import sys
sys.path.append('..')
from connectors.msgraph import Msgraph
mg = Msgraph()

In [2]:
# Tranform list of tuples to dict
def tuple_to_dict(tuple_list:list[tuple[str,Any]])->dict[str, Any]:
    return dict((key, value) for key, value in tuple_list)

# Get all apps from devices

**URL to get all the devices**

GET https://graph.microsoft.com/v1.0/deviceManagement/managedDevices/

## Get the first device which has the Windows OS

In [30]:
id = list(filter(lambda x: x["operatingSystem"]=="Windows", await mg.query("https://graph.microsoft.com/v1.0/deviceManagement/managedDevices/")))[0]["id"]
print(id)
id = "7fe91247-dea9-4acc-9940-70310f694af1"

MsGraph: https://graph.microsoft.com/v1.0/deviceManagement/managedDevices/ runned sucesfully
MsGraph: https://graph.microsoft.com/v1.0/deviceManagement/managedDevices/?$skiptoken=LastDeviceName%3d%27TSL-LP-4ZWSL33%27%2cLastDeviceId%3d%27084b3bf5-71cc-4883-9262-5dc256a840d8%27 runned sucesfully
e10ea415-dfb2-4ca2-9fd8-277741e3d40c


## Get all the APPS from the first windows device

### Using the MsGraph v1.0 🚫

In [24]:
await mg.query('https://graph.microsoft.com/v1.0/deviceManagement/managedDevices/{id}/detectedApps')

MsGraph: ERROR https://graph.microsoft.com/v1.0/deviceManagement/managedDevices/{id}/detectedApps failed
	BadRequest -- Resource not found for the segment 'detectedApps'.


[]

### Using the Msgraph Beta ✅

In [31]:
# Best solution for now to get the apps of the devices
await mg.query(f'https://graph.microsoft.com/beta/deviceManagement/managedDevices/{id}/detectedApps')

MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices/7fe91247-dea9-4acc-9940-70310f694af1/detectedApps runned sucesfully
MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices/7fe91247-dea9-4acc-9940-70310f694af1/detectedApps?$skip=50 runned sucesfully
MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices/7fe91247-dea9-4acc-9940-70310f694af1/detectedApps?$skip=100 runned sucesfully
MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices/7fe91247-dea9-4acc-9940-70310f694af1/detectedApps?$skip=150 runned sucesfully


[{'id': '0000b7171cc556a7b379d131c768ffb7fa7a0000ffff',
  'displayName': '1Password',
  'version': '8.10.24',
  'sizeInByte': 0,
  'deviceCount': 0,
  'publisher': '',
  'platform': 'windows'},
 {'id': '35907b2b9fa9f3dc1b484bd5494b9f8e9b6a570d5010636f1ec6b829873e2851',
  'displayName': '22490Automattic.Simplenote',
  'version': '2.21.0.0',
  'sizeInByte': 0,
  'deviceCount': 0,
  'publisher': 'CN=E2E5A157-746D-4B04-9116-ABE5CB928306',
  'platform': 'windows'},
 {'id': 'de7062c38cd1ec337657b3118d4fdc60f3b3aea8b6d42eb790c8e590d17dceb8',
  'displayName': '4505Fortemedia.FMAPOControl',
  'version': '1.0.38.0',
  'sizeInByte': 0,
  'deviceCount': 0,
  'publisher': 'CN=DC696A1A-C35B-41EC-9C89-CC577F52CD81',
  'platform': 'windows'},
 {'id': '3287e0723d6af204f3d22c220c7232f2207cde5c6a34f317ed44ba0f01208079',
  'displayName': '5319275A.WhatsAppDesktop',
  'version': '2.2401.5.0',
  'sizeInByte': 0,
  'deviceCount': 0,
  'publisher': 'CN=24803D75-212C-471A-BC57-9EF86AB91435',
  'platform': 'win

In [21]:
await mg.query(f'https://graph.microsoft.com/beta/deviceManagement/managedDevices/{id}')

MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices/7f1802c5-a7b4-4863-9e46-aaa64683ffc6 runned sucesfully


[]

## 🚫 Get all the detected apps from all devices with just one URL 

In [19]:
# get all decices with the detected apps -- NOT WORKING --
list(filter(lambda x: x["operatingSystem"]=="Windows" and x["detectedApps"] != [], await mg.query("https://graph.microsoft.com/beta/deviceManagement/managedDevices?$expand=detectedApps")))

MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices?$expand=detectedApps runned sucesfully
MsGraph: https://graph.microsoft.com/beta/deviceManagement/managedDevices?$expand=detectedApps&$skiptoken=LastDeviceName%3d%27TSL-LP-4ZWSL33%27%2cLastDeviceId%3d%27084b3bf5-71cc-4883-9262-5dc256a840d8%27 runned sucesfully


[]

⚠️ All the devices return a empty list of the apps they have.

## ✅ Get all the windows apps from all the devices, consulting from each device 

In [20]:
devices = list((filter(lambda x: x["operatingSystem"]=="Windows", await mg.query("https://graph.microsoft.com/v1.0/deviceManagement/managedDevices/"))))
fields = ["id", "deviceName", "userId", "userDisplayName", "emailAddress", "operatingSystem", "osVersion"]

devices = list(map(lambda x: tuple_to_dict(list(map(lambda y: (y, x[y]), fields))), devices))[:50]

# apps = list(map(lambda x: await mg.query(f'https://graph.microsoft.com/beta/deviceManagement/managedDevices/{x["id"]}/detectedApps'), devices))
apps = []
for index, device in enumerate(devices):
    id = device["id"]
    device_apps = await mg.query(f'https://graph.microsoft.com/beta/deviceManagement/managedDevices/{id}/detectedApps')
    for device_app in device_apps:
        device_app["deviceDetails"] = device
        apps.append(device_app)
print(apps)

CancelledError: 

Device fields
```py
{'id': 'e10ea415-dfb2-4ca2-9fd8-277741e3d40c',
 'userId': '1d7984d4-ba2f-4f98-800a-00b7a5dff323',
 'deviceName': 'DESKTOP-8TVV75J',
 'managedDeviceOwnerType': 'company',
 'enrolledDateTime': '2024-02-13T19:00:55Z',
 'lastSyncDateTime': '2024-02-13T20:35:50Z',
 'operatingSystem': 'Windows',
 'complianceState': 'compliant',
 'jailBroken': 'Unknown',
 'managementAgent': 'mdm',
 'osVersion': '10.0.19045.3930',
 'easActivated': False,
 'easDeviceId': '3EAD192E17525817885A674933310CA4',
 'easActivationDateTime': '0001-01-01T00:00:00Z',
 'azureADRegistered': True,
 'deviceEnrollmentType': 'windowsAzureADJoin',
 'activationLockBypassCode': None,
 'emailAddress': 'Manuela.Guerrero@tabacaleracigar.com',
 'azureADDeviceId': 'f1feceb6-d974-4db9-a0e3-0070623e3bce',
 'deviceRegistrationState': 'registered',
 'deviceCategoryDisplayName': 'Unknown',
 'isSupervised': False,
 'exchangeLastSuccessfulSyncDateTime': '0001-01-01T00:00:00Z',
 'exchangeAccessState': 'none',
 'exchangeAccessStateReason': 'none',
 'remoteAssistanceSessionUrl': None,
 'remoteAssistanceSessionErrorDetails': None,
 'isEncrypted': True,
 'userPrincipalName': 'Manuela.Guerrero@tabacaleracigar.com',
 'model': 'OptiPlex 3080',
 'manufacturer': 'Dell Inc.',
 'imei': '',
 'complianceGracePeriodExpirationDateTime': '9999-12-31T23:59:59Z',
 'serialNumber': '7CGMMH3',
 'phoneNumber': '',
 'androidSecurityPatchLevel': '',
 'userDisplayName': 'Guerrero, Manuela',
 'configurationManagerClientEnabledFeatures': None,
 'wiFiMacAddress': '',
 'deviceHealthAttestationState': None,
 'subscriberCarrier': '',
 'meid': '',
 'totalStorageSpaceInBytes': 255349227520,
 'freeStorageSpaceInBytes': 208705421312,
 'managedDeviceName': 'Manuela.Guerrero_Windows_2/13/2024_7:00 PM',
 'partnerReportedThreatState': 'unknown',
 'requireUserEnrollmentApproval': None,
 'managementCertificateExpirationDate': '2025-02-10T15:48:21Z',
 'iccid': None,
 'udid': None,
 'notes': None,
 'ethernetMacAddress': None,
 'physicalMemoryInBytes': 0,
 'deviceActionResults': []}
```

In [ ]:
import json


print(json.dumps(apps[0:10], indent=4))

[
    {
        "id": "0000857cb2cc316ab122bf3b7473f89152bf00000a04",
        "displayName": "Adobe Acrobat Reader DC - Espa\u00f1ol",
        "version": "15.007.20033",
        "sizeInByte": 0,
        "deviceCount": 0,
        "publisher": "",
        "platform": "windows",
        "deviceDetails": {
            "id": "e10ea415-dfb2-4ca2-9fd8-277741e3d40c",
            "deviceName": "DESKTOP-8TVV75J",
            "userId": "1d7984d4-ba2f-4f98-800a-00b7a5dff323",
            "userDisplayName": "Guerrero, Manuela",
            "emailAddress": "Manuela.Guerrero@tabacaleracigar.com",
            "operatingSystem": "Windows",
            "osVersion": "10.0.19045.3930"
        }
    },
    {
        "id": "00002b7618fefbc31a83c577ac911f4eb3940000ffff",
        "displayName": "Aplicaciones de Microsoft 365 para empresas - es-es",
        "version": "16.0.17231.20236",
        "sizeInByte": 0,
        "deviceCount": 0,
        "publisher": "",
        "platform": "windows",
        "deviceDe

# Get Microsoft Licenses

In [2]:
response = await mg.query("https://graph.microsoft.com/v1.0/subscribedSkus")

print(json.dumps(response[0], indent=2))

MsGraph: https://graph.microsoft.com/v1.0/subscribedSkus runned sucesfully
[
  {
    "accountName": "tabacaleracigarrow",
    "accountId": "1410a3bb-e96e-4aa9-88df-fbbea4077580",
    "appliesTo": "User",
    "capabilityStatus": "Enabled",
    "consumedUnits": 5,
    "id": "1410a3bb-e96e-4aa9-88df-fbbea4077580_c5928f49-12ba-48f7-ada3-0d743a3601d5",
    "skuId": "c5928f49-12ba-48f7-ada3-0d743a3601d5",
    "skuPartNumber": "VISIOCLIENT",
    "subscriptionIds": [
      "a4acabf6-b4af-4e6d-8fe9-bdd49e609b5d"
    ],
    "prepaidUnits": {
      "enabled": 5,
      "suspended": 0,
      "warning": 0,
      "lockedOut": 0
    },
    "servicePlans": [
      {
        "servicePlanId": "da792a53-cbc0-4184-a10d-e544dd34b3c1",
        "servicePlanName": "ONEDRIVE_BASIC",
        "provisioningStatus": "Success",
        "appliesTo": "User"
      },
      {
        "servicePlanId": "2bdbaf8f-738f-4ac7-9234-3c3ee2ce7d0f",
        "servicePlanName": "VISIOONLINE",
        "provisioningStatus": "Success"